In [1]:
from comet_ml import Experiment
experiment = Experiment(
    api_key="VQ78p7UKivJgj8TU2KWKkCJNj",
    project_name="airbnb",
    workspace="nram8122",
)
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  
import tensorflow as tf
import sys
sys.path.append('./src')
from funcs import *
import pandas as pd
import numpy as np
import seaborn as sns
  

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nram8122/airbnb/9463fb1d8bd94172ac7a3b287312aa9c



## Experiments are Logged on Comet Ml
1. Click on the link to see the outputs

In [2]:

# binary data is used for the Lasso Model
categorical_binary = pd.read_csv(r'./preprocessed/categorical_binary.csv', index_col =0)
categorical_raw = pd.read_csv(r'./preprocessed/categorical_raw.csv', index_col =0)
regression_data = pd.read_csv(r'./preprocessed/regression_transformed.csv', index_col =0)
regression_cols = regression_data.columns
index = pd.read_csv('./preprocessed/benchmark_datasets_index.csv', index_col =0)
price = pd.read_csv('./preprocessed/price.csv', index_col =0)

In [3]:
X_train, X_test, X_val, y_train, y_test, y_val = create_training_dataset(categorical_binary, regression_data,
                                                                         price, index, regression_cols,
                                                                         norm_reg = True)

In [4]:
regression_cols = ['host_response_rate', 'host_acceptance_rate',
       'transformed_review_scores_rating',
       'transformed_review_scores_accuracy',
       'transformed_review_scores_cleanliness',
       'transformed_review_scores_checkin',
       'transformed_review_scores_communication',
       'transformed_review_scores_location',
       'transformed_review_scores_value']

In [5]:
reducelr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, verbose=0,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=10, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)
callbacks = [reducelr, early_stopping]

# Here a simple MLP
Suggestions for improvement
1. Use a multi input MLP - one branch for regression and the other for classification
2. Use tensorflow probability for mean, std estimates - very easy use case

In [6]:
import tensorflow as tf
input_layer = tf.keras.layers.Input(X_train.shape[1:])
intermediate_layer = tf.keras.layers.Dense(32, activation ='sigmoid')(input_layer)
dropout = tf.keras.layers.Dropout(0.3)(intermediate_layer)
output = tf.keras.layers.Dense(1, activation ='linear')(intermediate_layer)

In [7]:
model = tf.keras.models.Model(input_layer, output)
model.compile(loss ='huber', optimizer ='adam', metrics =['mse'])


In [8]:
history = model.fit(X_train.values, y_train.values, validation_data =(X_test.values, y_test.values),
                    epochs =100, batch_size =100, verbose =2, callbacks = callbacks)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


COMET INFO: ignoring tensorflow summary log of metrics because of keras; set `comet_ml.loggers.tensorboard_logger.LOG_METRICS = True` to override


Epoch 1/100


48/48 - 0s - loss: 4.4010 - mse: 24.7599 - val_loss: 3.4400 - val_mse: 16.0852 - lr: 0.0010 - 377ms/epoch - 8ms/step


Epoch 2/100
48/48 - 0s - loss: 2.7633 - mse: 11.3745 - val_loss: 1.8755 - val_mse: 6.2214 - lr: 0.0010 - 45ms/epoch - 935us/step


...

Epoch 41/100
48/48 - 0s - loss: 0.0800 - mse: 0.1704 - val_loss: 0.0895 - val_mse: 0.1841 - lr: 2.0000e-04 - 44ms/epoch - 914us/step


Epoch 42/100
48/48 - 0s - loss: 0.0799 - mse: 0.1701 - val_loss: 0.0895 - val_mse: 0.1840 - lr: 2.0000e-04 - 44ms/epoch - 914us/step


Epoch 43/100
48/48 - 0s - loss: 0.0799 - mse: 0.1700 - val_loss: 0.0895 - val_mse: 0.1841 - lr: 2.0000e-04 - 44ms/epoch - 914us/step


Epoch 44/100


48/48 - 0s - loss: 0.0798 - mse: 0.1698 - val_loss: 0.0897 - val_mse: 0.1843 - lr: 2.0000e-04 - 43ms/epoch - 893us/step


In [9]:
val_metrics, y_hat_val = eval_metrics(X_val, y_val, model)
test_metrics, y_hat_val = eval_metrics(X_test, y_test, model)
val_metrics.to_csv('./results/MLP_hyperparameter_val.csv')
test_metrics.to_csv('./results/MLP_hyparameter_test.csv')